In [1]:
import numpy as np
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import root_mean_squared_error
from data_generation import get_data, g_0, m_0
from dml_algorithm import dml_ate

In [21]:
# XGBoost
import pickle
with open('results_dict.pkl', 'rb') as pickle_file:
    results_dict_ = pickle.load(pickle_file)

results_dict = {}

for N, results in results_dict_.items():
    rmses = results[-1]
    results_dict[N] = np.mean(rmses, axis=0)

print(results_dict)

{250: array([1.56527874, 1.91652426, 0.18219812]), 500: array([1.21668199, 1.46546163, 0.15362898]), 1000: array([0.94717333, 1.1181108 , 0.12635505]), 2000: array([0.75846577, 0.87028295, 0.10306294]), 4000: array([0.61785315, 0.68996007, 0.0835763 ]), 8000: array([0.51142757, 0.56260442, 0.06938977]), 16000: array([0.43729764, 0.47732874, 0.05567659])}


In [22]:
# ElasticNet
import pickle
with open('results_dict.pkl', 'rb') as pickle_file:
    results_dict_ = pickle.load(pickle_file)

results_dict = {}

for N, results in results_dict_.items():
    rmses = results[-1]
    results_dict[N] = np.mean(rmses, axis=0)

print(results_dict)

{250: array([0.81783588, 0.91132411, 0.15982491]), 500: array([0.6223609 , 0.67033547, 0.12340524]), 1000: array([0.51418182, 0.54255521, 0.09347865]), 2000: array([0.4562664 , 0.47172106, 0.06874686]), 4000: array([0.42061945, 0.43096792, 0.04985691]), 8000: array([0.40363964, 0.41099026, 0.03545416]), 16000: array([0.39323008, 0.399441  , 0.02546802])}


In [2]:
rng = np.random.default_rng(seed=42)

In [22]:
N = 2000
y_data, d_data, x_data = get_data(N, rng)
scaler = StandardScaler()
x_data_stand = scaler.fit_transform(x_data)
y_train, y_test, d_train, _, _, x_test, x_train_stand, x_test_stand = train_test_split(y_data, d_data, x_data, x_data_stand, test_size=0.2, random_state=42)

In [28]:
%%time
model_g = MLPRegressor(hidden_layer_sizes=(16,8), alpha=0.1, batch_size=4, max_iter=100, random_state=42)
d = 0
for i in range(50):
    model_g.partial_fit(x_train_stand[d_train==d], y_train[d_train==d])
    print('Iteration', i, root_mean_squared_error(g_0(d, x_test), model_g.predict(x_test_stand)))

Iteration 0 3.0871461733010883
Iteration 1 2.3496800724272284
Iteration 2 1.848339871056931
Iteration 3 1.3607841321505854
Iteration 4 1.0831166307169349
Iteration 5 0.9543203494647527
Iteration 6 0.8854397110977299
Iteration 7 0.840597266521013
Iteration 8 0.8071884547876211
Iteration 9 0.7802596106094232
Iteration 10 0.7542329091676118
Iteration 11 0.7295376252168759
Iteration 12 0.7069829836544957
Iteration 13 0.6875528980914516
Iteration 14 0.6691490737482388
Iteration 15 0.6490236121170432
Iteration 16 0.6337827567009665
Iteration 17 0.6144053752855914
Iteration 18 0.6001025738542558
Iteration 19 0.5848976122471188
Iteration 20 0.5742389221918033
Iteration 21 0.5629204212330442
Iteration 22 0.5536366042538625
Iteration 23 0.5425379458787867
Iteration 24 0.5330110179096063
Iteration 25 0.5255924822334963
Iteration 26 0.5163789143706037
Iteration 27 0.5085414057851628
Iteration 28 0.5042700694881136
Iteration 29 0.49672374508763667
Iteration 30 0.49211432500698404
Iteration 31 0.487

In [31]:
model_g = MLPRegressor(solver='lbfgs', hidden_layer_sizes=(32,16,8), alpha=0.1, batch_size=64, max_iter=10000, random_state=42)
model_g.fit(x_train_stand[d_train==1], y_train[d_train==1])

C:\Users\henry\MA_CausalML\dml\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


MLPRegressor(alpha=0.1, batch_size=64, hidden_layer_sizes=(32, 16, 8),
             max_iter=10000, random_state=42, solver='lbfgs',
             validation_fraction=0)

In [30]:
root_mean_squared_error(g_0(0, x_test), model_g.predict(x_test_stand))

0.4441553684936346

In [51]:
%%time
model_m = MLPClassifier(hidden_layer_sizes=(32,26), alpha=0.2, batch_size=16, max_iter=50, random_state=42)
for i in range(50):
    model_m.partial_fit(x_train_stand, d_train, [0,1])
    print('Iteration', i, root_mean_squared_error(m_0(x_test), model_m.predict_proba(x_test_stand)[:,1]))

Iteration 0 0.21805351708206464
Iteration 1 0.15409318186955603
Iteration 2 0.11763740530976427
Iteration 3 0.09765423402095888
Iteration 4 0.08599570300106892
Iteration 5 0.07896084695930923
Iteration 6 0.07404842457128684
Iteration 7 0.07004802342634094
Iteration 8 0.06688004459537458
Iteration 9 0.06411694783911533
Iteration 10 0.06174452821934631
Iteration 11 0.06002435217155167
Iteration 12 0.058769541242460824
Iteration 13 0.05758098208865844
Iteration 14 0.05655767431570276
Iteration 15 0.05586749845371985
Iteration 16 0.05531012997455993
Iteration 17 0.05482874221102371
Iteration 18 0.054541673922953046
Iteration 19 0.05453422043277819
Iteration 20 0.05429814864139204
Iteration 21 0.05399970788703717
Iteration 22 0.05375452448335419
Iteration 23 0.05352752340024101
Iteration 24 0.05335699855478055
Iteration 25 0.05298159861596445
Iteration 26 0.05273352235472647
Iteration 27 0.05237464433407297
Iteration 28 0.05214196693571484
Iteration 29 0.051978363889980075
Iteration 30 0.05

In [31]:
%%time
model_m = MLPClassifier(hidden_layer_sizes=(32,32,16), alpha=0.1, batch_size=4, max_iter=50, random_state=42)
model_m.fit(x_train_stand, d_train)

CPU times: total: 11.6 s
Wall time: 11.7 s


MLPClassifier(alpha=0.1, batch_size=4, hidden_layer_sizes=(32, 32, 16),
              max_iter=50, random_state=42)

In [60]:
root_mean_squared_error(m_0(x_test), model_m.predict_proba(x_test_stand)[:,1])

0.14982370553476335

In [52]:
N = 250
y_data, d_data, x_data = get_data(N, rng)
y_train, y_test, d_train, d_test, x_train, x_test = train_test_split(y_data, d_data, x_data, test_size=0.2, random_state=42)
model_m = CalibratedClassifierCV(estimator=LinearSVC(dual='auto', max_iter=2500))
model_m_2 = CalibratedClassifierCV(estimator=LinearSVC(dual='auto', max_iter=2500), method='isotonic')

In [53]:
%%time
model_m.fit(x_train, d_train)
print(model_m.predict_proba(x_test)[:20,1])
model_m_2.fit(x_train, d_train)
print(model_m_2.predict_proba(x_test)[:20,1])
print(m_0(x_test[:20]))

[0.75782366 0.1081392  0.4380588  0.13226565 0.22477492 0.32252746
 0.52142833 0.55220563 0.3364116  0.61823012 0.76432348 0.29846205
 0.46542094 0.15033277 0.65025305 0.4393908  0.85164284 0.57002081
 0.63447986 0.04004093]
[0.93333333 0.         0.4512605  0.         0.09203475 0.30238095
 0.4512605  0.4912605  0.33952381 0.67239993 0.93333333 0.27507003
 0.4912605  0.05714286 0.72466527 0.4512605  0.96012949 0.54226408
 0.62380952 0.        ]
[0.8485524  0.09058125 0.30348166 0.02835181 0.13868599 0.39836533
 0.46815584 0.49492956 0.23402002 0.82336364 0.70270092 0.35112246
 0.48414538 0.03198229 0.78706381 0.32661086 0.83445733 0.65141855
 0.6412689  0.00732812]
CPU times: total: 109 ms
Wall time: 115 ms
